In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

import numpy as np
import cv2

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir = 'train'
val_dir = 'test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode='grayscale',
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),padding='valid', activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [5]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/10


C:\Users\User\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\User\AppData\Local\Temp\ipykernel_4180\1454336883.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_info = model.fit_generator(


448/448 [==============================] - 42s 92ms/step - loss: 1.8612 - accuracy: 0.3408 - val_loss: 1.7106 - val_accuracy: 0.3387
Epoch 2/10
448/448 [==============================] - 34s 76ms/step - loss: 1.4693 - accuracy: 0.4392 - val_loss: 1.3954 - val_accuracy: 0.4622
Epoch 3/10
448/448 [==============================] - 34s 76ms/step - loss: 1.3292 - accuracy: 0.4981 - val_loss: 1.3353 - val_accuracy: 0.4978
Epoch 4/10
448/448 [==============================] - 34s 76ms/step - loss: 1.2234 - accuracy: 0.5405 - val_loss: 1.3011 - val_accuracy: 0.5060
Epoch 5/10
448/448 [==============================] - 35s 79ms/step - loss: 1.1211 - accuracy: 0.5865 - val_loss: 1.2629 - val_accuracy: 0.5251
Epoch 6/10
448/448 [==============================] - 34s 76ms/step - loss: 1.0259 - accuracy: 0.6222 - val_loss: 1.2506 - val_accuracy: 0.5301
Epoch 7/10
448/448 [==============================] - 35s 78ms/step - loss: 0.9270 - accuracy: 0.6649 - val_loss: 1.2198 - val_accuracy: 0.5420
Epo

In [7]:
from keras.models import load_model

model.save('model67_73.h5')

In [1]:
import cv2
import numpy as np
from keras.models import load_model

# Load trained model
emotion_model = load_model("model67_73.h5")

# Dictionary mapping class labels with corresponding emotions
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Happy", 3: "Fearful", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Load the image
image = cv2.imread('C:/Users/user/Desktop/smile.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find haar cascade to draw bounding box around face
face_cascade = cv2.CascadeClassifier('C:/Users/User/Desktop/haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
    roi_gray = gray[y:y + h, x:x + w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
    prediction = emotion_model.predict(cropped_img)
    max_index = int(np.argmax(prediction))
    emotion_label = emotion_dict[max_index]
    cv2.putText(image, emotion_label, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Display the image
cv2.imshow('Emotion Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

exit(0)


1/1 [==============================] - 0s 12ms/step
